<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
from itertools import product

1. Подберите скорость обучения (alpha) и количество итераций:

In [2]:
# 1. Функции

def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err


# запускает решетчатый поиск по alpha_range и n_iter_range
def simple_grad_search(X, y_true, W_init, alpha_range, n_iters_range, verbose=True, verbose_step=100):
    cases = list(product(n_iters_range, alpha_range))
    N = len(y_true)
    err_list = []
    W_list = []
    param_list = []
    for case in cases:
        for i in range(case[0]):
            W = W_init
            y_pred = np.dot(W, X)
            err = calc_mse(y_true, y_pred)
            for ii in range(W.shape[0]):
                W[ii] -= case[1] * (1/N * 2 * np.sum(X[ii] * (y_pred - y_true)))
                if verbose and i % verbose_step == 0:
                    print(i, W, err)
        err_list.append(err)
        W_list.append(W)
        param_list.append(case)
    min_error = np.array(err_list).min()
    idx = np.array(err_list).argmin()
    best_param = param_list[idx]
    best_W = W_list[idx]
    print(f'best params: learning rate {best_param[1]} and n_iter {best_param[0]} with mse = {min_error}')
    return best_W    

In [3]:
# 2. Входные данные
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2]])

y_true = [45, 55, 50, 59, 65, 35, 75, 80, 50, 60]

In [4]:
# 3. Запуск поиска параметров
W_init = np.array([1, 0.5])
alpha_range = [1e-2, 1e-3, 1e-4, 1e-5]
n_iters_range = [1000, 2000, 3000, 5000]
w_optimum = simple_grad_search(X, y_true, W_init, alpha_range, n_iters_range, 
                               verbose=False, verbose_step=100)

w_optimum

best params: learning rate 0.01 and n_iter 2000 with mse = 45.9375


array([47.23214286,  3.91071429])

In [5]:
# 4. Проверка результатов на единичном коде
alpha = 1e-2
n_iter = 2000

n = X.shape[1]
W = np.array([1, 0.5])

for i in range(2000):
    y_pred = np.dot(W, X)
    err = calc_mse(y_true, y_pred)
    for ii in range(W.shape[0]):
        W[ii] -= alpha * (1/n * 2 * np.sum(X[ii] * (y_pred - y_true)))
        if i % 500 == 0:
            print(i, W, err)

0 [2.102 0.5  ] 3173.15
0 [2.102 3.9  ] 3173.15
500 [46.99760587  3.95447839] 45.96769776787538
500 [46.99760587  3.95402334] 45.96769776787538
1000 [47.23088237  3.91094949] 45.937500872219864
1000 [47.23088237  3.91094704] 45.937500872219864
1500 [47.23213608  3.91071555] 45.937500000025196
1500 [47.23213608  3.91071554] 45.937500000025196


*2. В этом коде мы избавляемся от итераций по весам, но тут есть ошибка, исправьте ее:

In [8]:
# 1. Error

# ошибка в расчетax коэффициентов
# суть ошибки в np.sum(), т.к. в случае применения к матрице - возвращается не вектор размерности W, а скаляр

In [10]:
# Исправленный вариант
W = np.array([1, 0.5])
alpha = 1e-2
for i in range(2000):
    y_pred = np.dot(W, X)
    err = calc_mse(y_true, y_pred)
    W = W - alpha * (1/n * 2 * np.sum(X * (y_pred - y_true), axis=1)) # добавлено уточнение по оси
    #print(W)
    if i % 200 == 0:
        print(i, W, err)   

0 [2.102 3.9  ] 3173.15
200 [41.83683774  4.90699865] 61.9177717428135
400 [46.56511152  4.03388672] 46.181755648107604
600 [47.14967657  3.92594232] 45.941233404700036
800 [47.2219474   3.91259695] 45.93755706443538
1000 [47.23088237  3.91094704] 45.937500872219864
1200 [47.23198702  3.91074306] 45.93750001333172
1400 [47.23212359  3.91071784] 45.937500000203784
1600 [47.23214048  3.91071473] 45.93750000000313
1800 [47.23214256  3.91071434] 45.93750000000003


Задание *3: вместо того, чтобы задавать количество итераций, задайте условие остановки алгоритма - когда ошибка за итерацию
начинает изменяться ниже определенного порога (упрощенный аналог параметра tol в линейной регрессии в sklearn).

In [11]:
# версия функции c остановкой по tol, реализована без цикла по параметрам
def single_grad_tol(X, y_true, W_init, alpha, tol_level, verbose=True, verbose_step=100):
    N = len(y_true)
    y_pred = np.dot(W_init, X)
    start_err = calc_mse(y_true, y_pred)
    W = W_init
    idx = 0
    while True:
        idx += 1
        W -= alpha * (1/n * 2 * np.sum(X * (y_pred - y_true), axis=1))
        y_pred = np.dot(W, X)
        current_err = calc_mse(y_true, y_pred)
        tol = start_err - current_err
        start_err = current_err
        if tol <= tol_level:
            print(f'tolerance level reached on {idx} iter')
            return W
        else:
            if verbose and idx % verbose_step == 0:
                print(idx, W, err)
            continue
        

In [12]:
W_init = np.array([1, 0.5])
alpha = 1e-2
tol_level = 1e-15

w_optimum_tol = single_grad_tol(X, y_true, W_init, alpha, tol_level, verbose=True, verbose_step=300)

w_optimum_tol

300 [45.3151501   4.26470166] 45.937500000000014
600 [47.14881009  3.92610232] 45.937500000000014
900 [47.22852033  3.91138321] 45.937500000000014
1200 [47.23198538  3.91074336] 45.937500000000014
1500 [47.23213601  3.91071555] 45.937500000000014
tolerance level reached on 1629 iter


array([47.23214108,  3.91071461])